### ===> TASK MANAGER AGENT <===

##### Step 1 - Importing Required Libraries

In [3]:
import spacy  # NLP Library to extract meaningful info from natural language
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer   # Converts text into numeric format
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import sqlite3
import os
import dateparser   # Parses date strings into datetime object
from datetime import datetime, date

##### Step 2 - Load spaCy Model

In [5]:
# !pip install spacy download en_core_web_sm
# !python -m spacy download en_core_web_md
nlp = spacy.load("en_core_web_md")  ## Medium sized pre-trained english model

##### Step 3 - Creating/loading Task Data

In [59]:
csv = "Task_Manager.csv"  # Creating a csv file 
if not os.path.exists(csv) :
    sample = [
        ("Submit final report by tomorrow", "Work",	"High"),
        ("Attend daily stand-up meeting today",	"Work",	"Medium"),
        ("Organize your desk this weekend",	"Work",	"Low"),
        ("Take grandma to hospital this Friday", "Personal", "High"),
        ("Go for a morning walk tomorrow", "Personal", "Medium"),
        ("Watch a movie with family tonight", "Personal", "Low"),
        ("Pay electricity bill before next Monday",	"Errand", "High"),
        ("Drop off package at courier by Friday", "Errand",	"Medium"),
        ("Buy cleaning supplies this evening", "Errand", "Low"),
        ("Study for math test on Saturday",	"Learning",	"High"),
        ("Complete ML tutorial by next week", "Learning", "Medium"),
        ("Read a textbook chapter tonight", "Learning", "Low"),
        ("Prepare slides by end of today", "Work", "High"),
        ("Review project goals this afternoon", "Work", "Medium"),
        ("Sort emails tomorrow", "Work", "Low"),
        ("Help dad with doctor visit on Monday", "Personal", "High"),
        ("Call your best friend this evening", "Personal", "Medium"),
        ("Organize phone gallery this weekend", "Personal",	"Low"),
        ("Renew driver's license by Friday", "Errand", "High"),
        ("Pick up vegetables tomorrow morning", "Errand", "Medium"),
        ("Replace batteries before Sunday", "Errand", "Low"),
        ("Revise notes for quiz by tomorrow night", "Learning", "High"),
        ("Watch recorded class this weekend", "Learning", "Medium"),
        ("Practice typing for 10 mins today", "Learning", "Low") ]

    df = pd.DataFrame(sample, columns = ["Description", "Category", "Priority"])
    df.to_csv(csv, index = False)
else :
    df = pd.read_csv(csv)

##### Displaying the Data Frame created with Sample Task Descriptions to get trained 

In [9]:
df.head()

,Description,Category,Priority
0,Submit final report by tomorrow,Work,High
1,Attend daily stand-up meeting today,Work,Medium
2,Organize your desk this weekend,Work,Low
3,Take grandma to hospital this Friday,Personal,High
4,Go for a morning walk tomorrow,Personal,Medium


##### Step 4 - Training ML Model (for "Category" and "Priority")

In [61]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["Description"])

# Categorical Model
X_train_cat, _, y_train_cat, _ = train_test_split(X, df["Category"], test_size = 0.2, random_state = 42)
cat_model = LogisticRegression(max_iter = 1000)
cat_model.fit(X_train_cat, y_train_cat)

# Priority Model
X_train_pri, _, y_train_pri, _ = train_test_split(X, df["Priority"], test_size = 0.2, random_state = 42)
pri_model = LogisticRegression(max_iter = 1000)
pri_model.fit(X_train_pri, y_train_pri)

LogisticRegression(max_iter=1000)

##### Step 5 - Extract Deadlines

In [13]:
from dateutil.relativedelta import relativedelta  # Relative date math (Compute Date)

def extract_deadline(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "DATE":
            # Manual adjustments for simple keywords
            keyword = ent.text.strip().lower()
            today = datetime.now()
            if keyword == "today":
                return today.strftime("%Y-%m-%d")
            elif keyword == "tomorrow":
                return (today + relativedelta(days=1)).strftime("%Y-%m-%d")
            elif keyword == "next week":
                return (today + relativedelta(weeks=1)).strftime("%Y-%m-%d")
            elif keyword == "next month":
                return (today + relativedelta(months=1)).strftime("%Y-%m-%d")
            else:
                parsed = dateparser.parse(ent.text, settings={"RELATIVE_BASE": today})
                if parsed:
                    return parsed.strftime("%Y-%m-%d")
    parsed = dateparser.parse(text, settings={"RELATIVE_BASE": datetime.now()})
    if parsed:
        return parsed.strftime("%Y-%m-%d")
    return "None"

##### Step 6 - Create Task Database

In [15]:
conn = sqlite3.connect("Tasks_Man.db")
cursor = conn.cursor()
cursor.execute("""CREATE TABLE IF NOT EXISTS Tasks
        (id INTEGER PRIMARY KEY,
        description TEXT,
        category TEXT,
        priority TEXT,
        deadline TEXT)""")
conn.commit()

##### Step 7 - Building a Function to Retrain Models after New Tasks

In [17]:
def retrain_models() :
    print("Retraining Models with New Data.................................................")

    # Reload the task data
    df = pd.read_csv(csv)

    # Re Train the ML Model with Updated Data
    X = vectorizer.fit_transform(df["Description"])

    # Re-Train Category Model 
    X_train_cat, _, y_train_cat, _ = train_test_split(X, df["Category"], test_size=0.2, random_state=42)
    cat_model.fit(X_train_cat, y_train_cat)

    # Re-Train Priority Model
    X_train_pri, _, y_train_pri, _ = train_test_split(X, df["Priority"], test_size=0.2, random_state=42)
    pri_model.fit(X_train_pri, y_train_pri)

    print("Models Re-Trained Successfully!")

##### Step 8 - Adding a Task

In [19]:
def add_task(description) :
    # ML Prediction
    vector = vectorizer.transform([description])
    category = cat_model.predict(vector)[0]
    priority = pri_model.predict(vector)[0]

    # spaCy based deadline extraction
    deadline = extract_deadline(description)

    # Save to CSV (Learning Menory)
    pd.DataFrame([[description, category, priority]], columns = ["Description", "Category", "Priority"]).to_csv(csv,  mode = 'a', header = False, index = False) 

    # Save to Database
    cursor.execute("INSERT INTO Tasks (description, category, priority, deadline) VALUES (?, ?, ?, ?)", (description, category, priority, deadline))
    conn.commit()

    # Retrain After New Tasks Added
    retrain_models()

    # OutPut
    print("\n Task Added")
    print(f" Description : {description}")
    print(f" Category : {category}")
    print(f" Priority : {priority}")
    print(f" Deadline : {deadline}")

##### Step 9 - Displaying Tasks

In [21]:
def view_tasks():
    cursor.execute("SELECT * FROM Tasks")
    rows = cursor.fetchall()
    if not rows:
        print("No tasks yet. Add some using add_task().")
    else:
        print("\n📋 All Tasks:")
        today = date.today()
        for row in rows:
            deadline_str = row[4]
            if deadline_str and deadline_str != "None":
                try:
                    deadline_date = datetime.strptime(deadline_str, "%Y-%m-%d").date()
                    days_diff = (deadline_date - today).days
                    if days_diff == 0:
                        status = "(Due Today)"
                    elif days_diff < 0:
                        status = "(Overdue)"
                    else:
                        status = f"(Due in {days_diff} days)"
                except:
                    status = ""
            else:
                status = ""

            print(f"[{row[0]}] {row[1]} | Category: {row[2]} | Priority: {row[3]} | Deadline: {row[4]} {status}")

##### Step 10 - Function to Delete all Tasks from the Database

In [23]:
def delete_all_tasks():
    cursor.execute("DELETE FROM tasks")
    conn.commit()
    print("All tasks have been deleted from the database.")

## ------------------------------------------------------------------------------------------------------------

## ----------------------------=> Adding Tasks To The Task Manager <=----------------------------------

##### -> New Tasks

In [73]:
# add_task("Wish my mother on her birthday on May 30")

In [75]:
# add_task("Finish the project by tomorrow")

In [77]:
# view_tasks()

In [79]:
# delete_all_tasks()

In [81]:
# view_tasks()